In [2]:
import torch
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F
import torch.optim as optim
from torch import nn
import matplotlib.pyplot as plt
from utils import load_nerl_data, get_normalized_adj, get_Laplace, calculate_random_walk_matrix,test_error_cap
import random
import copy
import scipy
from scipy.io import loadmat
import math
import pandas as pd
from basic_structure import D_GCN, C_GCN, K_GCN,IGNNK

Define the parameter for NREL case

In [3]:
n_o_n_m = 100 #sampled space dimension

h = 16 #sampled time dimension

z = 100 #hidden dimension for graph convolution

K = 1 #If using diffusion convolution, the actual diffusion convolution step is K+1

n_m = 30 #number of mask node during training

N_u = 30 #target locations, N_u locations will be deleted from the training data

Max_episode = 750 #max training episode

learning_rate = 0.0001 #the learning_rate for Adam optimizer

batch_size = 8

STmodel = IGNNK(h, z, K)

STmodel = torch.load('model/nrel_ignnk_sigmaA_cap_20210621.pth')

Load data

As we further explored IGNNK, we found that adjacency matrix definition is crucial for the performance of GNN traning. Especially in the definition of sigma in Gaussian kernel based adjacency. Here we use a Gaussian Process based method to find an optimal sigma. 

In [4]:
# _, X, files_info = load_nerl_data()
X = np.load('data/nrel/nerl_X.npy')
files_info = pd.read_pickle('data/nrel/nerl_file_infos.pkl')
dist_mx = loadmat('data/nrel/nrel_dist_mx_lonlat.mat')
dist_mx = dist_mx['nrel_dist_mx_lonlat']
dis = dist_mx/1e3
A = np.exp( -0.5* np.power( dis/14 ,2) )

# We only use 7:00am to 7:00pm as the target data, because otherwise the 0-values of periods without sunshine will greatly influence the results
time_used_base = np.arange(84,228)
time_used = np.array([])
for i in range(365):
    time_used = np.concatenate((time_used,time_used_base + 24*12* i))
X=X[:,time_used.astype(np.int)]

capacities = np.array(files_info['capacity'])
capacities = capacities.astype('float32')
E_maxvalue = capacities.max()

X = X.transpose()/capacities
print(X.shape,E_maxvalue)


split_line1 = int(X.shape[0] * 0.7)
training_set = X[:split_line1, :]
test_set = X[split_line1:, :]  # split the training and test period

rand = np.random.RandomState(0) # Fixed random output, just an example when seed = 0.
unknow_set = rand.choice(list(range(0,X.shape[1])),N_u,replace=False)
unknow_set = set(unknow_set)
full_set = set(range(0,X.shape[1]))
know_set = full_set - unknow_set
training_set_s = training_set[:, list(know_set)]   # get the training data in the sample time period
A_s = A[:, list(know_set)][list(know_set), :]      # get the observed adjacent matrix from the full adjacent matrix,
                                                   # the adjacent matrix are based on pairwise distance, 
                                                   # so we need not to construct it for each batch, we just use index to find the dynamic adjacent matrix

(52560, 137) 100.0


Output the result from the best IGNNK model

In [5]:
MAE_t, RMSE_t, R2_t, nrel_ignnk_res  = test_error_cap(STmodel, unknow_set, full_set,test_set, A,h,capacities)
print('Best model', MAE_t, RMSE_t, R2_t)

Best model 1.7373738777085028 2.7775878816683157 0.9355863602349819
